<a href="https://colab.research.google.com/github/UniVR-DH/ADHLab/blob/main/lecture01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Wikipedia Python APIs to create a document collection

<img src="https://drive.google.com/uc?export=view&id=1m_EMdnI5C826kgqK7r5vB4TXnB0-Wq7W" alt="Intestazione con loghi istituzionali" width="525"/>

| Docente      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Matteo Lissandrini      | Laboratorio Avanzato di Informatica Umanistica       | 2023/2024   |

### Installing additional packages

In [5]:
%pip install wikipedia-api
%pip install beautifulsoup4

### Importing some required packages

In [6]:
import gzip
import string
import numpy as np

In [26]:
import requests
import regex as re
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/New_York_City')

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

# Pull text from all instances of <p> tag within BodyText div
all_p_items = soup.find(class_='mw-body').find_all('p')
print(len(all_p_items))
print(all_p_items[0])
print(all_p_items[1])

161
<p class="mw-empty-elt">
</p>
<p><b>New York</b>, often called <b>New York City</b><sup class="reference" id="cite_ref-12"><a href="#cite_note-12">[b]</a></sup> or simply <b>NYC</b>, is the <a href="/wiki/List_of_United_States_cities_by_population" title="List of United States cities by population">most populous city</a> in the <a href="/wiki/United_States" title="United States">United States</a>. With a population of 8,804,190 distributed over 300.46 square miles (778.2 km<sup>2</sup>) in 2020, the city is the <a href="/wiki/List_of_United_States_cities_by_population_density" title="List of United States cities by population density">most densely populated</a> major city in the United States. NYC is more than twice as populous as <a href="/wiki/Los_Angeles" title="Los Angeles">Los Angeles</a>, the nation's second-most populous city. New York City is at the southern tip of <a href="/wiki/New_York_(state)" title="New York (state)">New York State</a> and is situated on <a href="/wiki

In [36]:
punct_regex = re.compile('[{}]'.format(re.escape(string.punctuation))) # Regex matching any punctuation
space_regex = re.compile(' +') # Regex matching whitespace

text = punct_regex.sub(' ', soup.get_text())
text = space_regex.sub(' ', text).lower()  # convert to lowercase
lines = [

    line.strip()
    for line in text.split("\n")
    if line.strip() != "" # Skip empty lines
]
# Store lines
print(lines[0])

NameError: ignored

In [34]:
all_a_items = soup.find(class_='mw-body').find_all('a')
print(len(all_a_items))
for a in all_a_items:
  href = a.get('href')
  if href is not None and href.startswith('/wiki/') :
    print(href)

6088
/wiki/New_York_City
/wiki/Talk:New_York_City
/wiki/New_York_City
/wiki/New_York_City
/wiki/Special:WhatLinksHere/New_York_City
/wiki/Special:RecentChangesLinked/New_York_City
/wiki/Wikipedia:File_Upload_Wizard
/wiki/Special:SpecialPages
/wiki/Geographic_coordinate_system
/wiki/New_York_City_(disambiguation)
/wiki/NYC_(disambiguation)
/wiki/New_York,_New_York_(disambiguation)
/wiki/City_(New_York)
/wiki/File:View_of_Empire_State_Building_from_Rockefeller_Center_New_York_City_dllu_(cropped).jpg
/wiki/Midtown_Manhattan
/wiki/Empire_State_Building
/wiki/Lower_Manhattan
/wiki/One_World_Trade_Center
/wiki/File:67%C2%BA_Per%C3%ADodo_de_Sesiones_de_la_Asamblea_General_de_Naciones_Unidas_(8020913157)_(cropped).jpg
/wiki/Headquarters_of_the_United_Nations
/wiki/File:Liberty02.jpg
/wiki/Statue_of_Liberty
/wiki/File:New_york_times_square-terabass_(cropped).jpg
/wiki/Times_Square
/wiki/File:Unisphere_at_night_(cropped).jpg
/wiki/Unisphere
/wiki/File:20170721_Gotham_Shield_NYC_Aerials-225_mediu

In [ ]:
######
# TODO: Create a dictionary of /wiki/ links, and count how many times they appear in the page, which are the top-5 most frequent links?
######

In [7]:
import wikipediaapi
wapi_text = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                                   'en',
                                   extract_format=wikipediaapi.ExtractFormat.WIKI)

In [17]:
page_py = wapi_text.page('New York City')
print("Page - Exists: {}".format( page_py.exists()))
print(len(page_py.summary))
print(len(page_py.text))
print(len(page_py.langlinks))
print(len(page_py.links))

Page - Exists: True
5150
92129
247
2648


In [19]:
print(page_py.summary[:140])
print("   ---   ")
print(page_py.text[-140:])
print("   ---   ")
print(sorted(page_py.langlinks.keys()))
print("   ---   ")




New York, often called New York City or simply NYC, is the most populous city in the United States. With a population of 8,804,190 distribut
   ---   
 145,000 NYC photographs at the Museum of the City of New York
"The New New York Skyline (interactive)". National Geographic. November 2015.
   ---   
['af', 'als', 'am', 'an', 'ang', 'ar', 'arc', 'ary', 'arz', 'as', 'ast', 'awa', 'ay', 'az', 'azb', 'ba', 'ban', 'bar', 'bat-smg', 'bcl', 'be', 'be-x-old', 'bg', 'bh', 'bi', 'bjn', 'bm', 'bn', 'bo', 'br', 'bs', 'bug', 'bxr', 'ca', 'cbk-zam', 'cdo', 'ce', 'ceb', 'ch', 'ckb', 'co', 'crh', 'cs', 'cu', 'cv', 'cy', 'da', 'dag', 'de', 'diq', 'dsb', 'dty', 'ee', 'el', 'eml', 'eo', 'es', 'et', 'eu', 'ext', 'fa', 'fi', 'fiu-vro', 'fj', 'fo', 'fr', 'frp', 'frr', 'fur', 'fy', 'ga', 'gag', 'gan', 'gcr', 'gd', 'gl', 'glk', 'gn', 'got', 'gu', 'gv', 'ha', 'hak', 'he', 'hi', 'hif', 'hr', 'hsb', 'ht', 'hu', 'hy', 'hyw', 'ia', 'id', 'ie', 'ig', 'ik', 'ilo', 'inh', 'io', 'is', 'it', 'ja', 'jam', 'jbo', 'jv', 

In [ ]:
links = page_py.links
for title in sorted(links.keys()):
    print("%s: %s" % (title, links[title]))


In [ ]:
test_pages = ['Addis Ababa',  'Tom Sawyer', 'Johannes Gutenberg', 'Marcel Proust', 'Google', ' Statoil']
languages = set(['als','en', 'es', 'fo', 'fr', 'da','de', 'hu', 'pl', 'pt', 'ru', 'nl', 'nov'])


In [ ]:
wapi_html = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                              'en',
                              extract_format=wikipediaapi.ExtractFormat.HTML)
